# EM Algorithm

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_digits
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [3]:
digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.3, random_state=10)

X_train /= 16
X_test /= 16

In [4]:
np.shape(X_train)

(1257, 64)

# Gaussian NB

In [5]:
class GaussianNbc():
    def __init__(self, seed = 10, epsilon = 1e-3):
        self.seed = seed
        self.epsilon = epsilon

    def gaussian_prob(self, x, mu, sigma_squared):
        return 1/np.sqrt(2*np.pi*sigma_squared) * np.exp(-1/(2*sigma_squared) * (x-mu) ** 2)

    def fit(self, X, K):
        self.K = K
        np.random.seed(self.seed)

        n_samples, n_features = np.shape(X)

        priors = np.ones(K)/K


        # Create subsamples of classes ie. creating n_samples of indices of all classes and shuffle.
        # The samples are randomly assigned to a class
        idxs = np.resize(range(K), n_samples)
        np.random.shuffle(idxs)
        
        mu = np.empty([K, n_features])
        sigma_squared = np.empty([K, n_features])

        # Calculate the mu and the sqaured sigma (variance) of each class
        for k in range(K):
            mu[k] = np.mean(X[idxs == k], axis = 0)
            sigma_squared[k] = np.var(X[idxs == k], axis = 0)

        sigma_squared += self.epsilon
        
        P = np.empty([n_samples, K])
        r = np.empty([n_samples, K])

        prev_prior = np.zeros(K)
        
        while np.linalg.norm(prev_prior - priors) > 1e-6:
            # E step
            for k in range(K):
                P[:, k] = np.prod(self.gaussian_prob(X, mu[k], sigma_squared[k]), axis=1)

            r = priors * P / (np.sum(priors * P, axis = 1)).reshape(-1, 1)

            # M step
            r_k = np.sum(r, axis = 0)
            prev_prior = priors

            priors = r_k / n_samples

            for k in range(K):
                mu[k] = np.sum(r[:, k].reshape(-1,1) * X, axis = 0) / r_k[k]
                # Our X is a row vector so we need to transpose it and perform xT * x to have a resulting matrix instead of a scalar
                sigma_squared[k] = np.diag( ( r[:, k].reshape(-1, 1) * (X - mu[k]) ).T @ (X - mu[k]) / r_k[k] )

            sigma_squared += self.epsilon
        self.mu = mu
        self.sigma_squared = sigma_squared
        self.priors = priors

    def predict(self, X):
        preds = np.empty(len(X))
        prob = np.zeros(self.K)

        for i, x in enumerate(X):
            for k in range(self.K):
                prob[k] = self.priors[k] * np.prod(self.gaussian_prob(x, self.mu[k], self.sigma_squared[k]))
            
            preds[i] = np.argmax(prob)

        self.preds = preds
        return self.preds


In [6]:
clustering = GaussianNbc()
clustering.fit(X_train, 10)

In [7]:
predictions = clustering.predict(X_test)

In [8]:
print("Completeness score: %s" %(metrics.completeness_score(y_test, predictions)))
print("Homogeneity score: %s" %(metrics.homogeneity_score(y_test, predictions)))
print("AMI score: %s" %(metrics.adjusted_mutual_info_score(y_test, predictions)))

Completeness score: 0.6821415438378211
Homogeneity score: 0.6542738047882036
AMI score: 0.6557012219396601


# K Means

In [9]:
class KMeansEM():
    def __init__(self, seed=10, epsilon=1e-4):
        self.seed = seed
        self.epsilon = epsilon
    
    def fit(self, X, K):
        self.K = K
        np.random.seed(self.seed)
        priors = np.ones(K)/K

        n_samples, n_features = np.shape(X)
        
        idxs = np.resize(range(K), n_samples)
        np.random.shuffle(idxs)

        # Initialise mu
        mu = np.empty([K, n_features])
        for k in range(K):
            mu[k] = np.mean(X[idxs == k], axis=0)


        prev_mu = np.zeros([K, n_features])

        while np.sum(np.linalg.norm(mu - prev_mu)) > 1e-6:
            # E step
            for i, x in enumerate(X):
                cluster_distance = np.empty(K)
                for k in range(K):
                    cluster_distance[k] = np.linalg.norm(x - mu[k])
                x_cluster = np.argmin(cluster_distance, axis=0)
                idxs[i] = x_cluster

            # M step
            prev_mu = mu
            for k in range(K):
                mu[k] = np.mean(X[idxs == k], axis=0)
        
        self.mu = mu
    
    def predict(self, X):
        preds = np.empty(len(X))
        cluster_distance = np.zeros(self.K)

        for i, x in enumerate(X):
            for k in range(self.K):
                cluster_distance[k] = np.linalg.norm(x - self.mu[k])
            preds[i] = np.argmin(cluster_distance, axis=0)

        self.preds = preds
        return self.preds
        

In [10]:
kmeans = KMeansEM()
kmeans.fit(X_train, 10)

In [11]:
preds = kmeans.predict(X_test)

In [12]:
print("Homogeneity score: %s" %metrics.homogeneity_score(y_test, preds))
print("Completeness score: %s" %metrics.completeness_score(y_test, preds))
print("AMI score: %s" %metrics.adjusted_mutual_info_score(y_test, preds))

Homogeneity score: 0.5451246898456436
Completeness score: 0.5615544031918106
AMI score: 0.5369016946840789


# K Means Scikit

In [13]:
from sklearn.cluster import KMeans
sk_kmeans = KMeans(10)
sk_kmeans.fit(X_train)
preds = sk_kmeans.predict(X_test)

In [14]:
print("Homogeneity score: %s" %metrics.homogeneity_score(y_test, preds))
print("Completeness score: %s" %metrics.completeness_score(y_test, preds))
print("AMI score: %s" %metrics.adjusted_mutual_info_score(y_test, preds))

Homogeneity score: 0.739391403680697
Completeness score: 0.7540718109012448
AMI score: 0.7375062058724509
